In [1]:
! which python

/scratch-data/wu1396/condaenv/py37/bin/python


In [2]:
import torch

In [3]:
!pwd

/scratch-data/wu1396/complex-order_oct_16_2021/TEXT_CLASSIFICATION/Experiment_BERT


In [4]:
!nvidia-smi

Wed Oct 20 14:57:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:18:00.0 Off |                  N/A |
| 25%   25C    P8     8W / 250W |      1MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [205]:
# model_name = "bhadresh-savani/distilbert-base-uncased-emotion"
# model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# model_name = "aychang/distilbert-base-cased-trec-coarse"

# model_name = "textattack/bert-base-uncased-SST-2"
# model_name = "aychang/bert-base-cased-trec-coarse"
# model_name = "textattack/bert-base-uncased-ag-news"
# model_name = "bhadresh-savani/bert-base-uncased-emotion"

# model_name = "textattack/roberta-base-SST-2"
model_name = "bhadresh-savani/roberta-base-emotion"

In [206]:
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name, cache_dir="./cache/model").to("cuda:0")
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/983 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/983 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [158]:
from datasets import load_dataset
# dataset = load_dataset('trec', split='test', cache_dir="./cache") #
# dataset = load_dataset('glue', 'sst2', split='validation', cache_dir="./cache") #
# dataset = load_dataset('ag_news', split='test', cache_dir="./cache") #
dataset = load_dataset('emotion', split='test', cache_dir="./cache") #

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/204k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/207k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset emotion downloaded and prepared to ./cache/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705. Subsequent calls will reuse this data.


In [110]:
# from datasets import get_dataset_config_names
# configs = get_dataset_config_names("ag_news")

Downloading:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

In [119]:
# dataset.info

In [120]:
# dataset.split, dataset.description, dataset.citation, dataset.homepage

AttributeError: 'DatasetDict' object has no attribute 'split'

In [159]:
# dataset[0], dataset[1], dataset[2]

({'label': 0,
  'text': 'im feeling rather rotten so im not very ambitious right now'},
 {'label': 0, 'text': 'im updating my blog because i feel shitty'},
 {'label': 0,
  'text': 'i never make her separate from me because i don t ever want her to feel like i m ashamed with her'})

In [10]:
# dataset.shape, dataset.num_columns, dataset.num_rows, len(dataset)

In [160]:
dataset.column_names

['text', 'label']

In [15]:
# dataset.features

In [15]:
# dataset.features['label'].num_classes, dataset.features['label'].names

In [16]:
# dataset[:3], dataset[[1, 3, 5]]

In [12]:
# encoded_dataset = dataset.map(lambda examples: tokenizer(examples['text'], truncation=True, padding='max_length'), batched=True)
# encoded_dataset = encoded_dataset.rename_column('label-coarse','labels')
# encoded_dataset.column_names
# encoded_dataset

In [13]:
# set(list(encoded_dataset['labels']))

In [162]:
from datasets import load_metric
metric = load_metric('accuracy')

In [15]:
# print(metric.inputs_description)

In [138]:
from tqdm import tqdm
import random 
import os
import time
import numpy as np
from functools import partial

In [128]:
def assign_GPU(Tokenizer_output):

    tokens_tensor = Tokenizer_output['input_ids'].to('cuda:0')
    labels = Tokenizer_output['labels'].to('cuda:0')
    attention_mask = Tokenizer_output['attention_mask'].to('cuda:0')

    output = {'input_ids' : tokens_tensor, 
              'labels' : labels, 
              'attention_mask' : attention_mask}

    return output

In [129]:
from operator import itemgetter

def add_prefix(example):
    index = torch.cat([torch.randperm(sum(example['attention_mask'])), torch.arange(sum(example['attention_mask']), len(example['input_ids']))])
    example['input_ids'] = list(itemgetter(*index.tolist())(example['input_ids']))
    return example

In [19]:
# encoded_dataset = dataset.map(lambda examples: tokenizer(examples['text'], truncation=True, padding='max_length'), batched=True)
# encoded_dataset = encoded_dataset.rename_column('label-coarse','labels')
# encoded_dataset.column_names
# # encoded_dataset[0]
# encoded_dataset = encoded_dataset.map(add_prefix, load_from_cache_file=False)
# encoded_dataset[0]['input_ids'][:15]

In [20]:
# encoded_dataset[0]['input_ids'][:15]

In [189]:
def swap_random(example, num_scramble=7):
    idx = range(sum(example['attention_mask']))
    for _i in range(num_scramble):
        i1, i2 = random.sample(idx, 2)
        example['input_ids'][i1], example['input_ids'][i2] = example['input_ids'][i2], example['input_ids'][i1]
    return example
    

In [190]:
def swap_random_neighbors(example, num_scramble=7):
    idx = range(sum(example['attention_mask']))
    for _i in range(num_scramble):
        i1 = random.sample(idx, 1)[0]
        i2 = (i1+1)%len(idx)
        example['input_ids'][i1], example['input_ids'][i2] = example['input_ids'][i2], example['input_ids'][i1]
    return example
    

In [191]:
def set_seed(seed=int(time.time())):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed()

In [192]:
def process_data(num_scramble, shuffle_type="none"):
    encoded_dataset = dataset.map(lambda examples: tokenizer(examples['text'], truncation=True, padding='max_length'), batched=True)
    encoded_dataset = encoded_dataset.rename_column('label','labels')
#     print(encoded_dataset.column_names)
    # encoded_dataset[0]

    if shuffle_type == "all":
        encoded_dataset = encoded_dataset.map(add_prefix, load_from_cache_file=False)
    elif shuffle_type == "any":
        encoded_dataset = encoded_dataset.map(partial(swap_random, num_scramble=num_scramble), load_from_cache_file=False)
    elif shuffle_type == "neighbors":
        encoded_dataset = encoded_dataset.map(partial(swap_random_neighbors, num_scramble=num_scramble), load_from_cache_file=False)

    print(encoded_dataset[0]['input_ids'][:15])



    encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
#     encoded_dataset = assign_GPU(encoded_dataset)
    return torch.utils.data.DataLoader(encoded_dataset, batch_size=32)

In [164]:
def eval_test(dataloader):
    pt_model.eval()
    for batch in tqdm(dataloader):
        batch = {k: v.to("cuda:0") for k, v in batch.items()}
        with torch.no_grad():
            outputs = pt_model(**batch)

        logits = outputs.logits
#         print(logits[:5])
        predictions = torch.argmax(logits, dim=-1)
    #     print(predictions.shape)
    #     print(batch["labels"].shape)
        metric.add_batch(predictions=predictions, references=batch["labels"])
#         print(logits[:5])

    res = metric.compute()
    print(res)
    return res

In [165]:
print(pt_model.distilbert.embeddings.word_embeddings.weight)  #.embeddings.word_embeddings.weight

Parameter containing:
tensor([[-0.01665, -0.06660, -0.01633,  ..., -0.01999, -0.05139, -0.02635],
        [-0.01320, -0.06732, -0.01605,  ..., -0.02266, -0.05536, -0.02600],
        [-0.01759, -0.07093, -0.01443,  ..., -0.02457, -0.05955, -0.02318],
        ...,
        [-0.02310, -0.05877, -0.01049,  ..., -0.01945, -0.02615, -0.02118],
        [-0.04901, -0.05614, -0.00465,  ..., -0.01065, -0.01797, -0.02187],
        [-0.00646, -0.09147, -0.00255,  ..., -0.01505, -0.05039,  0.04597]],
       device='cuda:0', requires_grad=True)


In [212]:
metrics = {}
NUM_RUNS = 5
NUM_SCRAMBLE = [1, 3, 5, 7]  # 
SHUFFLE_TYPE = ["any", "neighbors"]  # "all"

for _st in SHUFFLE_TYPE:
    metrics[_st] = {}
    for _ns in NUM_SCRAMBLE:
        metrics[_st][_ns] = []
        for _j in range(NUM_RUNS):
        #     set_seed(_j)
            dataloader = process_data(_ns, _st)
            res = eval_test(dataloader)
            metrics[_st][_ns].append(res)

Loading cached processed dataset at ./cache/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-3c03f8e49c2bba8c.arrow


  0%|          | 0/2000 [00:00<?, ?ex/s]

[0, 757, 2157, 4356, 34485, 98, 1195, 45, 182, 8263, 235, 122, 2, 1, 1]


100%|██████████| 63/63 [00:33<00:00,  1.87it/s]
Loading cached processed dataset at ./cache/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-3c03f8e49c2bba8c.arrow


{'accuracy': 0.898}


...

{'accuracy': 0.891}


  0%|          | 0/2000 [00:00<?, ?ex/s]

[2157, 757, 0, 1195, 98, 34485, 4356, 45, 8263, 182, 235, 122, 2, 1, 1]


100%|██████████| 63/63 [00:34<00:00,  1.83it/s]

{'accuracy': 0.8855}


In [213]:
torch.set_printoptions(5)

for k1, v1 in metrics.items():
    for k2, v2 in v1.items():
        accs = [acc['accuracy'] for acc in v2]
        print(f"{k1}, {k2}: {torch.mean(torch.tensor(accs))} ({torch.std(torch.tensor(accs))})")

any, 1: 0.9012999534606934 (0.006310704164206982)
any, 3: 0.85670006275177 (0.0033279138151556253)
any, 5: 0.8220000267028809 (0.004636809229850769)
any, 7: 0.7996999621391296 (0.006369836162775755)
neighbors, 1: 0.9264000058174133 (0.0028592010494321585)
neighbors, 3: 0.9161999821662903 (0.002539675682783127)
neighbors, 5: 0.9023000001907349 (0.0038177191745489836)
neighbors, 7: 0.8904999494552612 (0.005396754946559668)


In [214]:
accs

[0.8955, 0.896, 0.8845, 0.891, 0.8855]

In [ ]:
# DistilBERT, Emotion
# Original: 0.927
# Sw1: 89.756 (0.415)
# Sw3: 84.780 (0.807)
# Sw5: 81.990 (0.498)
# Sw7: 79.660 (1.021)
# Shuffle: 74.320 (0.537)

# Sc1: 92.360 (0.198)
# Sc3: 90.770 (0.355)
# Sc5: 89.370 (0.590)
# Sc7: 88.490 (0.507)

In [ ]:
# BERT-base, Emotion
# Original: 92.650
# Sw1: 89.630 (0.506)
# Sw3: 84.860 (0.953)
# Sw5: 81.450 (0.939)
# Sw7: 79.300 (1.137)
# Shuffle: 74.850 (0.663)

# Sc1: 92.030 (0.164)
# Sc3: 90.620 (0.220)
# Sc5: 89.340 (0.763)
# Sc7: 88.360 (0.574)

In [ ]:
# RoBERTa, Emotion
# Original: 93.050
# Sw1: 90.130 (0.631)
# Sw3: 85.670 (0.333)
# Sw5: 82.200 (0.464)
# Sw7: 79.970 (0.637)
# Shuffle: 75.760 (0.934)

# Sc1: 92.640 (0.286)
# Sc3: 91.620 (0.254)
# Sc5: 90.230 (0.382)
# Sc7: 89.050 (0.540)

In [ ]:
# Bert-base, AG News
# Original: 0.951
# Sw1: 0.9496579170227051 (0.0013210464967414737)
# Sw3: 0.9443947672843933 (0.001222346443682909)
# Sw5: 0.9418420791625977 (0.001404880196787417)
# Sw7: 
# Shuffle: 0.9110790491104126 (0.0010080909123644233)

# Sc1: 
# Sc3: 
# Sc5: 
# Sc7: 

In [ ]:
# Bert-base, TREC
# Original: 97.400
# S1: 86.800 (1.386)
# S3: 76.480 (2.524)
# S5: 71.040 (0.932)
# S7: 70.200 (1.612)
# Shuffle: 67.520 (1.775)

# S1: 87.160 (1.526)
# S3: 79.600 (1.288)
# S5: 76.280 (1.323)
# S7: 74.960 (1.062)


In [ ]:
# DistilBert, TREC
# Original: 97.000
# S1: 88.560 (0.684)
# S3: 80.000 (1.463)
# S5: 76.000 (2.615)
# S7: 72.360 (0.740)
# Shuffle: 70.640 (2.220)

# S1: 90.840 (1.670)
# S3: 84.320 (1.006)
# S5: 80.640 (0.713)
# S7: 78.400 (1.435)


In [ ]:
# DistilBert, SST-2
# Sramb1: 89.908 (0.421)
# Sramb3: 89.564 (0.556)
# Sramb5: 88.761 (0.873)
# Sramb7: 88.165 (0.718)


In [ ]:
# Bert-base, SST-2
# Sramb1: 91.720 (0.469)
# Sramb3: 89.977 (0.330)
# Sramb5: 89.450 (0.506)
# Sramb7: 88.073 (0.573)


In [ ]:
# RoBERTa, SST-2
# Sramb1: 93.624 (0.131)
# Sramb3: 92.408 (0.534)
# Sramb5: 92.133 (0.192)
# Sramb7: 91.858 (0.579)


In [66]:
# BERT-base, Canonical
# 0.9243119266055045
# BERT-base, Shuffle
# 0.7821100917431193
# 0.7809633027522935
# 0.7763761467889908
# 0.7889908256880734
# 0.7821100917431193
# 0.78211 (0.00451)

# RoBERTa, Canonical
# 0.9403669724770642
# RoBERTa, Shuffle
# 0.8084862385321101
# 0.8096330275229358
# 0.8096330275229358
# 0.8004587155963303
# 0.8027522935779816
# 0.80619 (0.00429)

# DistilBERT, Canonical
# 0.9105504587155964

# DistilBERT, Shuffle
# 0.7844036697247706
# 0.7855504587155964
# 0.7752293577981652
# 0.768348623853211
# 0.7534403669724771
# 0.77339 (0.01319)

In [271]:
torch.set_printoptions(5)
torch.mean(torch.tensor([0.9197247706422018,0.9220183486238532,.9254587155963303,.9197247706422018,.9243119266055045]))


tensor(0.92225)

In [ ]:
# DistilBERT, 1-Scramble
# 0.8887614678899083
# 0.8853211009174312
# 0.8944954128440367
# 0.8922018348623854
# 0.8979357798165137
# 0.89174(0.00491)
 
# BERT-base, 1-Scramble
# 0.9162844036697247
# 0.9025229357798165
# 0.9059633027522935
# 0.8830275229357798
# 0.911697247706422
# 0.90390 (0.01281)

# RoBERTa, 1-Scramble
# 0.9197247706422018
# 0.9220183486238532
# 0.9254587155963303
# 0.9197247706422018
# 0.9243119266055045
# 0.92225 (0.00262)

In [277]:
torch.set_printoptions(5)
torch.std(torch.tensor([0.8990825688073395,0.9036697247706422,.8864678899082569,.8990825688073395,.8887614678899083]))


tensor(0.00741)

In [ ]:
# DistilBERT, 3-Scramble
# 0.8589449541284404
# 0.8555045871559633
# 0.8612385321100917
# 0.8532110091743119
# 0.8589449541284404
# 0.85757 (0.00318)

# BERT-base, 3-Scramble
# 0.8669724770642202
# 0.863562091503268
# 0.8784403669724771
# 0.8818807339449541
# 0.8692660550458715
# 0.87202 (0.00780)

# RoBERTa, 3-Scramble
# 0.8990825688073395
# 0.9036697247706422
# 0.8864678899082569
# 0.8990825688073395
# 0.8887614678899083
# 0.89541 (0.00741)


In [283]:
torch.set_printoptions(5)
torch.mean(torch.tensor([0.8941798941798942,0.8715596330275229,.8692660550458715,.8681192660550459,.8876146788990825]))


tensor(0.87815)

In [ ]:
# DistilBERT, 5-Scramble
# 0.8451834862385321
# 0.823394495412844
# 0.8360091743119266
# 0.8440366972477065
# 0.8371559633027523
# 0.83716 (0.00870)

# BERT-base, 5-Scramble
# 0.8532110091743119
# 0.8440366972477065
# 0.8623853211009175
# 0.8509174311926605
# 0.8486238532110092
# 0.85183 (0.00680)

# RoBERTa, 5-Scramble
# 0.8941798941798942
# 0.8715596330275229
# 0.8692660550458715
# 0.8681192660550459
# 0.8876146788990825
# 0.87815 (0.01193)
